![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Core Concepts of Context Engineering

## Learning Objectives (15 minutes)
By the end of this notebook, you will understand:
1. **The 4 core components** of context engineering
2. **The context window constraint** - the fundamental limitation
3. **Static vs. dynamic context** - when to use each
4. **5 essential best practices** for effective context engineering

## Prerequisites
- Completed `01_overview_and_first_example.ipynb`
- Seen context engineering in action

---

## The 4 Core Components

Every context-aware AI system has these 4 components. Let's see them in the agent you built:

### 1. System Context (Static)

**What it is:** Instructions and knowledge that rarely change

**From your example:**
```python
system_prompt = """
You are a class scheduling assistant.  # ← Role definition

Available Courses:                      # ← Domain knowledge
- CS401: Machine Learning...

Help students with course planning.     # ← Behavior instructions
"""
```

**Includes:**
- Agent role and personality
- Business rules and policies
- Domain knowledge
- Available tools and functions

### 2. Memory (Dynamic)

**What it is:** Information that persists across interactions

**From your example:**
```python
student_context = """
Student Profile:
- Completed Courses: CS101, CS201  # ← Persistent user data
- Current GPA: 3.7
"""
```

**Two types:**
- **Working Memory:** Current conversation context
- **Long-term Memory:** User preferences, history, facts

### 3. Context Retrieval (Dynamic)

**What it is:** Relevant information retrieved based on the current query

**Example:**
```python
# User asks: "What ML courses are available?"
# System retrieves:
relevant_courses = [
    "CS401: Machine Learning Fundamentals",
    "CS501: Advanced Machine Learning",
    "CS502: Deep Learning"
]
```

**Sources:**
- Database queries
- Vector search (semantic similarity)
- API calls to external services
- File system searches

### 4. Tools (Dynamic)

**What it is:** Functions the AI can call to take actions or get information

**Examples:**
```python
def search_courses(query):
    """Search for courses matching the query"""
    # Implementation here
    
def check_prerequisites(course_id, student_id):
    """Check if student meets prerequisites"""
    # Implementation here
    
def enroll_student(course_id, student_id):
    """Enroll student in course"""
    # Implementation here
```

**Purpose:** Enable AI to interact with external systems and take actions

## The Context Window Constraint

**The fundamental limitation:** Every AI model has a maximum amount of text it can process at once.

### Understanding Token Limits

**Context Window = Maximum tokens per request**

| Model | Context Window | Approximate Words |
|-------|----------------|-------------------|
| GPT-4o | 128,000 tokens | ~96,000 words |
| GPT-4o-mini | 128,000 tokens | ~96,000 words |
| Claude 3.5 Sonnet | 200,000 tokens | ~150,000 words |

**Note:** 1 token ≈ 0.75 words in English

### What Competes for Space?

Every request must fit:

```
┌─────────────────────────────────────────┐
│     CONTEXT WINDOW (128K tokens)        │
├─────────────────────────────────────────┤
│ System Instructions       │ 2,000       │
│ Tool Definitions          │ 3,000       │
│ Conversation History      │ 4,000       │
│ Retrieved Context         │ 5,000       │
│ User Query                │ 500         │
│ Response Space            │ 4,000       │
├─────────────────────────────────────────┤
│ TOTAL USED               │ 18,500      │
│ REMAINING                │ 109,500     │
└─────────────────────────────────────────┘
```

### Why This Matters

**Everything scales:**
- More tools → More tokens used
- Longer conversations → More tokens used  
- More retrieved data → More tokens used
- Larger knowledge base → More tokens used

**Context engineering is optimization within constraints.**

### The Trade-off Principle

**Every token spent on one thing is a token NOT available for another.**

**Good context engineering asks:**
1. Is this information relevant to the current query?
2. Does including this improve response quality?
3. Is the improvement worth the token cost?

**All three must be "yes" or don't include it.**

## Static vs. Dynamic Context

Context comes in two fundamentally different forms:

### Static Context (Rarely Changes)

**Definition:** Context that's fixed in your code, same for all users

**Characteristics:**
- Written directly in application code
- Same for all users and sessions
- Changes require code deployment
- Always present, fixed token cost

**Examples:**
```python
# Static - hardcoded in your application
SYSTEM_PROMPT = """
You are a class scheduling agent.
Always be helpful and encouraging.
Never recommend more than 5 courses at once.
"""

BUSINESS_RULES = """
- Students need 120 credits to graduate
- Maximum 18 credits per semester
- Prerequisites must be completed first
"""
```

**When to use static:**
- ✅ Applies to ALL users equally
- ✅ Defines agent's role/personality
- ✅ Rarely changes (less than monthly)
- ✅ Must always be present

### Dynamic Context (Constantly Changes)

**Definition:** Context retrieved at runtime, specific to user/session/query

**Characteristics:**
- Stored in databases (Redis, vector stores)
- Different for each user/session/query
- Retrieved based on relevance
- Variable token usage

**Examples:**
```python
# Dynamic - retrieved at runtime
conversation_history = get_conversation(session_id)
user_profile = get_student_profile(user_id)
relevant_courses = search_courses(query, limit=5)
```

**When to use dynamic:**
- ✅ Specific to a user or session
- ✅ Needs to be personalized
- ✅ Changes frequently
- ✅ Comes from external sources

### Design Decision Framework

**Question: Should X be static or dynamic?**

| Information | Static or Dynamic | Why |
|-------------|-------------------|-----|
| "You are a scheduling agent" | Static | Universal role definition |
| "Student prefers online courses" | Dynamic | User-specific preference |
| "Never recommend >5 courses" | Static | Universal business rule |
| "Student completed CS101 on 2024-01-15" | Dynamic | User-specific event |
| Available tool definitions | Static | Same tools for all users |
| Search results for "ML courses" | Dynamic | Query-specific results |

## 5 Essential Best Practices

### 1. Start Simple, Add Complexity Gradually

**❌ Wrong approach:**
```python
# Trying to build everything at once
system = ComplexAgent(
    tools=[50_different_tools],
    memory=AdvancedMemorySystem(),
    retrieval=HybridRAGSystem(),
    # ... 20 more components
)
```

**✅ Right approach:**
```python
# Step 1: Basic agent
agent = BasicAgent(system_prompt)

# Step 2: Add one tool
agent.add_tool(search_courses)

# Step 3: Add memory
agent.add_memory(conversation_memory)

# Step 4: Add retrieval
agent.add_retrieval(course_database)
```

### 2. Measure Token Usage

**Always know your token consumption:**
```python
def count_tokens(text):
    """Count tokens in text (approximate)"""
    return len(text.split()) * 1.3  # Rough estimate

# Before sending request
total_tokens = (
    count_tokens(system_prompt) +
    count_tokens(conversation_history) +
    count_tokens(retrieved_context) +
    count_tokens(user_query)
)

print(f"Total tokens: {total_tokens}")
print(f"Percentage of limit: {total_tokens/128000*100:.1f}%")
```

### 3. Optimize for Relevance, Not Completeness

**❌ Include everything:**
```python
# Bad: Including all 500 courses
context = get_all_courses()  # 50,000 tokens!
```

**✅ Include what's relevant:**
```python
# Good: Including top 5 relevant courses
context = search_courses(query, limit=5)  # 1,000 tokens
```

### 4. Use Clear, Structured Prompts

**❌ Unclear structure:**
```python
prompt = "You help with classes and here are courses CS101 intro programming CS201 data structures and student Alice completed CS101 help her"
```

**✅ Clear structure:**
```python
prompt = """
ROLE: Class scheduling assistant

AVAILABLE COURSES:
- CS101: Intro to Programming
- CS201: Data Structures (Prerequisite: CS101)

STUDENT PROFILE:
- Name: Alice
- Completed: CS101

TASK: Help the student plan their next courses.
"""
```

### 5. Test and Iterate

**Context engineering is empirical - test everything:**

```python
# Test different approaches
test_queries = [
    "Can I take CS401?",
    "What ML courses are available?",
    "Plan my next semester"
]

for query in test_queries:
    response = agent.ask(query)
    print(f"Query: {query}")
    print(f"Response: {response}")
    print(f"Quality: {rate_response(response)}/5")
    print("---")
```

**Iterate based on results:**
- Poor responses → Add more context
- Token limit errors → Remove less relevant context
- Slow responses → Reduce context size
- Wrong actions → Improve tool descriptions

## Key Takeaways

### The 4 Core Components
1. **System Context** - Role, rules, domain knowledge (static)
2. **Memory** - Conversation history, user preferences (dynamic)
3. **Context Retrieval** - Relevant data based on query (dynamic)
4. **Tools** - Functions to take actions (dynamic)

### The Fundamental Constraint
- **Context window limits** everything you can include
- **Every token counts** - optimize for relevance
- **Trade-offs are inevitable** - choose what matters most

### Static vs. Dynamic
- **Static:** Universal, hardcoded, fixed cost
- **Dynamic:** Personalized, retrieved, variable cost
- **Design decision:** Universal info → static, personalized info → dynamic

### Best Practices
1. Start simple, add complexity gradually
2. Measure token usage
3. Optimize for relevance, not completeness
4. Use clear, structured prompts
5. Test and iterate

---

## What's Next?

Now that you understand the core concepts and constraints, you're ready to dive deep into implementation.

**Next: Context Types Deep Dive**

In the next notebook, you'll master each context type with detailed, hands-on examples:
- System Context: Role definition and domain knowledge
- User Context: Personal information and preferences
- Conversation Context: Memory and dialogue history
- Retrieved Context: Dynamic information from external sources

You'll build context management systems, measure performance impact, and design strategies for different scenarios.

---

**Continue to: `03_context_types_deep_dive.ipynb`**